In [11]:
import csv

with open('/Users/saikrishna/Downloads/cleaned_data.csv', 'r') as file:
    reader = csv.reader(file)
    usernames = [row[0] for row in reader]


In [12]:
from github import Github

# Replace 'GITHUB_TOKEN' with your actual GitHub token
g = Github('GITHUB_TOKEN')

In [13]:
user_data = []

for username in usernames:
    try:
        user = g.get_user(username)
        repos = user.get_repos()
        for repo in repos:
            contents = repo.get_contents("")
            while contents:
                file_content = contents.pop(0)
                if file_content.type == 'dir':
                    contents.extend(repo.get_contents(file_content.path))
                else:
                    # Process code files (e.g., .py, .js, .java)
                    if is_code_file(file_content.name):
                        code = file_content.decoded_content.decode('utf-8')
                        # Analyze the code using Cohere API
                        analysis = analyze_code_with_cohere(code)
                        user_data.append({
                            'username': username,
                            'language': detect_language(file_content.name),
                            'complexity': analysis['complexity'],
                            'syntax_quality': analysis['syntax_quality']
                        })
    except Exception as e:
        print(f"Error processing user {username}: {e}")


Error processing user Username: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest", "status": "404"}
Error processing user savolai: status_code: 400, body: {'message': 'too many tokens: total number of tokens in the prompt cannot exceed 4081 - received 17294. Try using a shorter prompt, or enabling prompt truncating. See https://docs.cohere.com/reference/generate for more details.'}


KeyboardInterrupt: 

In [6]:
import cohere

co = cohere.Client('COHERE_API')

def analyze_code_with_cohere(code):
    # Use Cohere's generation or embedding models to analyze code
    response = co.generate(
        model='command',
        prompt=f"Analyze the following code for complexity and syntax quality:\n\n{code}\n\nProvide metrics for complexity and syntax quality.",
        max_tokens=50,
        temperature=0.5,
        stop_sequences=["\n"]
    )
    # Parse the response to extract metrics
    return parse_cohere_response(response.generations[0].text)


In [7]:
import os

def is_code_file(filename):
    code_extensions = ['.py', '.js', '.java', '.cpp', '.c', '.cs', '.rb', '.go']
    return os.path.splitext(filename)[1] in code_extensions

def detect_language(filename):
    return os.path.splitext(filename)[1][1:]  # Returns extension without dot


In [10]:
def parse_cohere_response(response_text):
    # Implement parsing logic based on the response format
    # For example, extract numerical values from the text
    complexity = extract_complexity(response_text)
    syntax_quality = extract_syntax_quality(response_text)
    return {
        'complexity': complexity,
        'syntax_quality': syntax_quality
    }

def extract_complexity(text):
    # Implement regex or string parsing to find complexity score
    pass

def extract_syntax_quality(text):
    # Implement regex or string parsing to find syntax quality score
    pass



In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.DataFrame(user_data)
scaler = MinMaxScaler()

df[['complexity_norm', 'syntax_quality_norm']] = scaler.fit_transform(df[['complexity', 'syntax_quality']])
df['total_points'] = df['complexity_norm'] + df['syntax_quality_norm']


KeyError: "None of [Index(['complexity', 'syntax_quality'], dtype='object')] are in the [columns]"